In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
data = pd.read_csv('../footfall_735.csv')
data.head()

,Unnamed: 0.1,Unnamed: 0,DATE,day,month,bogo,paneer,guest,test,meal_type,footfall,max_possible_footfall
0,0,0,2023-07-29,Saturday,July,0,0,0,0,Dinner,48,89
1,1,1,2023-08-21,Monday,August,0,0,1,0,Breakfast,307,325
2,2,2,2023-05-16,Tuesday,May,0,0,0,0,Dinner,181,225
3,3,3,2023-06-08,Thursday,June,0,0,0,0,Dinner,80,120
4,4,4,2023-08-07,Monday,August,0,0,0,0,Dinner,52,100


In [5]:
# dropping Unnamed: 0 column
data.drop('Unnamed: 0.1', axis=1, inplace=True)
data.drop('Unnamed: 0', axis=1, inplace=True)
data.head()

,DATE,day,month,bogo,paneer,guest,test,meal_type,footfall,max_possible_footfall
0,2023-07-29,Saturday,July,0,0,0,0,Dinner,48,89
1,2023-08-21,Monday,August,0,0,1,0,Breakfast,307,325
2,2023-05-16,Tuesday,May,0,0,0,0,Dinner,181,225
3,2023-06-08,Thursday,June,0,0,0,0,Dinner,80,120
4,2023-08-07,Monday,August,0,0,0,0,Dinner,52,100


In [6]:
data['date'] = pd.to_datetime(data['DATE'])
data.head()

,DATE,day,month,bogo,paneer,guest,test,meal_type,footfall,max_possible_footfall,date
0,2023-07-29,Saturday,July,0,0,0,0,Dinner,48,89,2023-07-29
1,2023-08-21,Monday,August,0,0,1,0,Breakfast,307,325,2023-08-21
2,2023-05-16,Tuesday,May,0,0,0,0,Dinner,181,225,2023-05-16
3,2023-06-08,Thursday,June,0,0,0,0,Dinner,80,120,2023-06-08
4,2023-08-07,Monday,August,0,0,0,0,Dinner,52,100,2023-08-07


In [7]:
data.drop('DATE', axis=1, inplace=True)

In [8]:
data2 = data.drop('month', axis=1)

In [9]:
# one hot encoding for day of week
data2 = pd.get_dummies(data, columns=['day'])
# one hot encoding for meal_type 
data2 = pd.get_dummies(data2, columns=['meal_type'])

In [10]:
data2.head()

,month,bogo,paneer,guest,test,footfall,max_possible_footfall,date,day_Friday,day_Monday,day_Saturday,day_Sunday,day_Thursday,day_Tuesday,day_Wednesday,meal_type_Breakfast,meal_type_Dinner,meal_type_Lunch,meal_type_Snacks
0,July,0,0,0,0,48,89,2023-07-29,False,False,True,False,False,False,False,False,True,False,False
1,August,0,0,1,0,307,325,2023-08-21,False,True,False,False,False,False,False,True,False,False,False
2,May,0,0,0,0,181,225,2023-05-16,False,False,False,False,False,True,False,False,True,False,False
3,June,0,0,0,0,80,120,2023-06-08,False,False,False,False,True,False,False,False,True,False,False
4,August,0,0,0,0,52,100,2023-08-07,False,True,False,False,False,False,False,False,True,False,False
